# Example Lipidomics Data Analysis (API)
_(lipydomics version: 1.4.x)_

---
## 1) Initialize a Dataset
We will be using `example_raw.csv` as the raw data file for this work (the data is positive mode and has not been normalized). We first need to initialize a lipydomics dataset from the raw data:

In [4]:
from lipydomics.data import Dataset

dset = Dataset('example_raw.csv', esi_mode='pos')

dset  # take a look at the Dataset

Dataset(
	csv="example_raw.csv",
	esi_mode="pos",
	samples=16,
	features=3342,
	identified=False,
	normalized=False,
	rt_calibrated=False,
	ext_var=False,
	group_indices=None,
	stats={}
)

We have now done the bare minimum to load the data and we have a lipydomics dataset initialized. We can see from the overview that there are 16 samples and 3342 features in this dataset. 
  
---
## 2) Prepare the Dataset
### 2.1) Assign Groups
Currently, we have 16 samples in our dataset, but we have not provided any information on what groups they belong to. We will assign them now. 

In [5]:
# dictionary mapping group identifiers (str) to sample indices (int)
group_idx = {
    '0641': [0, 1, 2, 3],
    'geh': [4, 5, 6, 7],
    'sal': [8, 9, 10, 11],
    'wt': [12, 13, 14, 15]
}
dset.assign_groups(group_idx)

dset  # take a look at the Dataset

Dataset(
	csv="example_raw.csv",
	esi_mode="pos",
	samples=16,
	features=3342,
	identified=False,
	normalized=False,
	rt_calibrated=False,
	ext_var=False,
	group_indices={
		"0641": [0, 1, 2, 3]
		"geh": [4, 5, 6, 7]
		"sal": [8, 9, 10, 11]
		"wt": [12, 13, 14, 15]
	},
	stats={}
)

Now all of the samples have been assigned to one of four groups: `0641`, `geh`, `sal1`, and `wt`. These group IDs will be used later on when we select data or perform statistical analyses. 
  
### 2.2) Normalize Intensities
Currently, the feature intensities are only raw values. We are going to normalize them using weights derived from an external normalization factor (pellet masses), but we also have the option to normalize to the signal from an internal standard if desired. The normalization weights are in `weights.txt`, a simple text file with the weights for each sample, one per line (16 total).

In [6]:
from numpy import genfromtxt

weights = genfromtxt('weights.txt')
dset.normalize(weights)

dset  # take a look at the Dataset

Dataset(
	csv="example_raw.csv",
	esi_mode="pos",
	samples=16,
	features=3342,
	identified=False,
	normalized=True,
	rt_calibrated=False,
	ext_var=False,
	group_indices={
		"0641": [0, 1, 2, 3]
		"geh": [4, 5, 6, 7]
		"sal": [8, 9, 10, 11]
		"wt": [12, 13, 14, 15]
	},
	stats={}
)

If we look at the dataset overview we can see that we now have assigned all of our samples to groups and we have a table of normalized intensities.
  
  
### 2.3) Identify Lipids
Another dataset preparation step we can perform before diving in to the data analysis is identifying as many lipids as possible. There are multiple identification criteria that take into account theoretical and measured m/z, retention time, and/or CCS, all of which vary in the level of confidence in the identifications they yield. We will use an approach that tries the highest confidence identification criteria first, then tries others. 

In [7]:
from lipydomics.identification import add_feature_ids

tol = [0.025, 0.5, 3.0]  # tol must be a list
# identify features at the highest level possible
add_feature_ids(dset, tol, level='any')

dset  # take a look at the Dataset

Dataset(
	csv="example_raw.csv",
	esi_mode="pos",
	samples=16,
	features=3342,
	identified=2134,
	normalized=True,
	rt_calibrated=False,
	ext_var=False,
	group_indices={
		"0641": [0, 1, 2, 3]
		"geh": [4, 5, 6, 7]
		"sal": [8, 9, 10, 11]
		"wt": [12, 13, 14, 15]
	},
	stats={}
)

Using the `any` identification level and m/z, retention time, and CCS tolerances of 0.025 0.5 3.0, respectively, 2063 lipids were identified. Now the dataset is fully prepared and we can start performing statistical analyses and generating plots. 
  
  
---
## 3) Statistical Analyses and Plotting
### 3.1) Compute ANOVA P-value for All Groups
A common analysis performed on lipidomics data is calculating the p-value of each feature from an ANOVA using the intensities from all groups. This gives an indication of how the variance between groups compares to the variance within groups, and a significant p-value indicates that there is some significant difference in the intensities for a given feature between the different groups.

In [8]:
from lipydomics.stats import add_anova_p

# compute ANOVA between all groups with normalized data
add_anova_p(dset, ['0641', 'geh', 'sal', 'wt'], normed=True)

dset  # take a look at the Dataset

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/scipy/stats/stats.py:3349: RuntimeWarning: invalid value encountered in double_scalars
  f = msb / msw


Dataset(
	csv="example_raw.csv",
	esi_mode="pos",
	samples=16,
	features=3342,
	identified=2134,
	normalized=True,
	rt_calibrated=False,
	ext_var=False,
	group_indices={
		"0641": [0, 1, 2, 3]
		"geh": [4, 5, 6, 7]
		"sal": [8, 9, 10, 11]
		"wt": [12, 13, 14, 15]
	},
	stats={
		"ANOVA_0641-geh-sal-wt_normed"
	}
)

_* The above `RuntimeWarning` can be ignored in this case, it is caused by the presence of features that have all 0 intensities which gives a within-group variance of 0 and therefore causing devision by 0._ 
  
  
### 3.2) Pricipal Components Analysis (All Groups)
PCA is an untargeted analysis that gives an indication of the overall variation between samples, as well as the individual features that contribute to this variation. We will compute a 3-component PCA in order to assess the variance between groups in this dataset. 

In [9]:
from lipydomics.stats import add_pca3

# compute PCA between all groups with normalized data
add_pca3(dset, ['0641', 'geh', 'sal', 'wt'], normed=True)

dset  # take a look at the Dataset

Dataset(
	csv="example_raw.csv",
	esi_mode="pos",
	samples=16,
	features=3342,
	identified=2134,
	normalized=True,
	rt_calibrated=False,
	ext_var=False,
	group_indices={
		"0641": [0, 1, 2, 3]
		"geh": [4, 5, 6, 7]
		"sal": [8, 9, 10, 11]
		"wt": [12, 13, 14, 15]
	},
	stats={
		"ANOVA_0641-geh-sal-wt_normed"
		"PCA3_0641-geh-sal-wt_loadings_normed"
		"PCA3_0641-geh-sal-wt_projections_normed"
	}
)

Now we have computed the 3-component PCA, and we can see two new stats entries in our dataset: "PCA3_0641-geh-sal1-wt_projections_normed" and "PCA3_0641-geh-sal1-wt_loadings_normed". Now we can take a look at the projections in a plot. 

In [11]:
from lipydomics.plotting import scatter_pca3_projections_bygroup

# all groups normalized data
scatter_pca3_projections_bygroup(dset, ['0641', 'geh', 'sal', 'wt'], './', normed=True)

Now we can take a look at the plot (`PCA3_0641-geh-sal1-wt_projections_normed.png`).

<img src="PCA3_0641-geh-sal1-wt_projections_normed.png" width="400" align="left">
  
  
It looks like `geh` and `wt` separate along PC1 while `sal1` and `wt` separate along PC2, so these might be a couple of good pairwise comparisons to explore further. 



### 3.3) PLS-DA and Correlation on `wt` and `geh`
Partial least-squares discriminant analysis (PLS-DA) is an analysis that is similar to PCA, except it finds significant variance between two specified groups (_i.e._ it is a supervised analysis).

In [12]:
from lipydomics.stats import add_plsda

# compute PLS-DA between 'wt' and 'geh' using raw data
add_plsda(dset, ['wt', 'geh'], normed=True)

dset  # take a look at the Dataset

Dataset(
	csv="example_raw.csv",
	esi_mode="pos",
	samples=16,
	features=3342,
	identified=2134,
	normalized=True,
	rt_calibrated=False,
	ext_var=False,
	group_indices={
		"0641": [0, 1, 2, 3]
		"geh": [4, 5, 6, 7]
		"sal": [8, 9, 10, 11]
		"wt": [12, 13, 14, 15]
	},
	stats={
		"ANOVA_0641-geh-sal-wt_normed"
		"PCA3_0641-geh-sal-wt_loadings_normed"
		"PCA3_0641-geh-sal-wt_projections_normed"
		"PLS-DA_wt-geh_loadings_normed"
		"PLS-DA_wt-geh_projections_normed"
	}
)

Now we have computed the PLS-DA, and we can see two new stats entries in our dataset: "PLS-DA_geh-wt_projections_normed" and "PLS-DA_geh-wt_loadings_normed". Now we can take a look at the projections in a plot.

In [13]:
from lipydomics.plotting import scatter_plsda_projections_bygroup

# groups wt vs. geh, normalized data
scatter_plsda_projections_bygroup(dset, ['wt', 'geh'], './', normed=True)

Now we can take a look at the plot (`PLS-DA_projections_geh-wt_normed.png`).

<img src="PLS-DA_projections_geh-wt_normed.png" width="400" align="left">

As expected, `geh` and `wt` separate cleanly along component 1 corresponding to between group differences. The spread of both groups along component 2, related to intra-group variance, is similar between both groups indicating a similar amount of variance in both groups uncorrelated between them. A similar targeted analysis is the Pearson correlation coefficient between the two groups, which we need to calculate in order to produce an S-plot and tease out which lipid features are driving the separation between `geh` and `wt`.

In [14]:
from lipydomics.stats import add_2group_corr
from lipydomics.plotting import splot_plsda_pcorr_bygroup

# compute correlation between 'wt' and 'geh' using normalized data
add_2group_corr(dset, ['wt', 'geh'], normed=True)

# groups wt vs. geh, normalized data
splot_plsda_pcorr_bygroup(dset, ['wt', 'geh'], './', normed=True)

dset  # take a look at the Dataset

Dataset(
	csv="example_raw.csv",
	esi_mode="pos",
	samples=16,
	features=3342,
	identified=2134,
	normalized=True,
	rt_calibrated=False,
	ext_var=False,
	group_indices={
		"0641": [0, 1, 2, 3]
		"geh": [4, 5, 6, 7]
		"sal": [8, 9, 10, 11]
		"wt": [12, 13, 14, 15]
	},
	stats={
		"ANOVA_0641-geh-sal-wt_normed"
		"PCA3_0641-geh-sal-wt_loadings_normed"
		"PCA3_0641-geh-sal-wt_projections_normed"
		"PLS-DA_wt-geh_loadings_normed"
		"PLS-DA_wt-geh_projections_normed"
		"2-group-corr_wt-geh_normed"
	}
)

We can take a look at the plot that was generated (`S-Plot_geh-wt_normed.png`).

<img src="S-Plot_geh-wt_normed.png" width="400" align="left">

There appear to be several lipid features that drive separation between `geh` and `wt`, as indicated by the points in the lower left (red) and upper right (blue) corners of the plot. The last step is to export the data and manually inspect these significant features.


## 4) Export Dataset to Spreadsheet
We need to export our processed Dataset into a spreadsheet format so that we can more closely inspect the data and identify the lipid features that drive the separation that we identified between the `geh` and `wt` groups.

In [15]:
dset.export_xlsx('example.xlsx')

## 5) Examine Specific Lipids
Manual inspection of the data has revealed a handful of individual lipid species that differ significantly between `geh` and `wt`:

| abundant in | m/z | retention time | CCS | putative id | id level |
| :---: | :---: | :---: | :---: | :--- | :--- |
| `geh` | 874.7869 | 0.43 | 320.3 | TG(52:3)_[M+NH4]+ | meas_mz_ccs |
| `geh` | 878.8154 | 0.62 | 322.7 | TG(52:1)_[M+NH4]+ | meas_mz_ccs |
| `geh` | 848.7709 | 0.40 | 313.3 | TG(50:2)_[M+NH4]+ | theo_mz_ccs |
| `geh` | 605.5523 | 0.86 | 267.7 | DG(36:1)_[M+H-H2O]+ | theo_mz_ccs |
| `geh` | 591.5378 | 0.93 | 263.9 | DG(35:1)_[M+H-H2O]+ | theo_mz_ccs |
| `wt` | 496.3423 | 4.15 | 229.8 | LPC(16:0)_[M+H]+ | meas_mz_ccs |
| `wt` | 524.3729 | 4.08 | 235.1 | LPC(18:0)_[M+H]+ | meas_mz_ccs |
| `wt` | 810.6031 | 3.46 | 295.3 | PC(36:1)_[M+Na]+ | meas_mz_ccs |
| `wt` | 782.5729 | 3.50 | 290.5 | PG(35:0)_[M+NH4]+ | theo_mz_ccs |

### 5.1) Generate Plots for Significant Lipid Features
Now that we have identified some potentially significant lipid feautures, we need to generate some bar plots for comparison. To avoid clogging up our working directory, we will save the feature plots in the `features` directory. The m/z, retention time, and CCS values are all listed in `features.csv`, and we will use this to generate the barplots all at once.

In [16]:
from lipydomics.plotting import batch_barplot_feature_bygroup

# tight search tolerance
tol = [0.01, 0.1, 1.0]
batch_barplot_feature_bygroup(dset, ['wt', 'geh'], 'features/', 'features.csv', tolerance=tol)

Now we can look at all of the plots that have been generated in the `features/` directory.

#### __Abundant in `geh`__
<img src="features/bar_874.7869-0.43-320.3_geh-wt_normed.png" width="200">
<img src="features/bar_878.8154-0.62-322.7_geh-wt_normed.png" width="200">
<img src="features/bar_848.7709-0.40-313.3_geh-wt_normed.png" width="200">
<img src="features/bar_605.5523-0.86-267.7_geh-wt_normed.png" width="200">
<img src="features/bar_591.5378-0.93-263.9_geh-wt_normed.png" width="200">


#### __Abundant in `wt`__
<img src="features/bar_496.3423-4.15-229.8_geh-wt_normed.png" width="200">
<img src="features/bar_524.3729-4.08-235.1_geh-wt_normed.png" width="200">
<img src="features/bar_810.6031-3.46-295.3_geh-wt_normed.png" width="200">
<img src="features/bar_782.5729-3.50-290.5_geh-wt_normed.png" width="200">

### 5.2) Generate a Heatmap of TGs 
There seems to be an upregulation of TGs in `geh` relative to `wt`, so it might be nice to see if there are any large-scale trends among TGs as a lipid class between these groups. In order to make this comparison, we will need to compute another statistic: the Log2(fold-change) between the two groups.

In [17]:
from lipydomics.stats import add_log2fc
from lipydomics.plotting import heatmap_lipid_class_log2fc

# compute Log2(fold-change) with normalized data
add_log2fc(dset, ['wt', 'geh'], normed=True)

# Lipid Class: TG
# Groups: wt, geh
# Save in Dir: ./
# Data: normalized
found_lipids = heatmap_lipid_class_log2fc('TG', dset, ['wt', 'geh'], './', normed=True)

# save the Dataset in a serialized binary format in case we want to load again later
dset.save_bin('example.pickle')

dset  # take a look at the Dataset

Dataset(
	csv="example_raw.csv",
	esi_mode="pos",
	samples=16,
	features=3342,
	identified=2134,
	normalized=True,
	rt_calibrated=False,
	ext_var=False,
	group_indices={
		"0641": [0, 1, 2, 3]
		"geh": [4, 5, 6, 7]
		"sal": [8, 9, 10, 11]
		"wt": [12, 13, 14, 15]
	},
	stats={
		"ANOVA_0641-geh-sal-wt_normed"
		"PCA3_0641-geh-sal-wt_loadings_normed"
		"PCA3_0641-geh-sal-wt_projections_normed"
		"PLS-DA_wt-geh_loadings_normed"
		"PLS-DA_wt-geh_projections_normed"
		"2-group-corr_wt-geh_normed"
		"LOG2FC_wt-geh_normed"
	}
)

We can take a look at the plot that was generated (`TG_geh-wt_log2fc_normed.png`).

<img src="TG_geh-wt_log2fc_normed.png" width="600" align="left">


